In [1]:
#regexp:
# #test([^\n]*)\n([^#][^\n]*)\n
# #test\1\n#test  \2\n

#
# класс узел
#   type
#   парсится     
#   отображается __repr__
#   op
#   args
#   vars
#
# класс утверждения
#   отображение: узел
#   внут. структ. : узел
#     получается из отображения
#
# класс доказанного утверждепия
#   утверждение//rel
#   получение из доказанного
#     подстановкой
#     C1
#     совпадает внут.структ.
#     S5
#     S6
#     S7
#     S8
#     создание аксиомы
#   ... ссылки на предков и потомков
#  
# автоматический поиск доказательства
#   ...
#
#
#
#
def typeop(t,o):
    def dec(nc):
        @property
        def type(self):
            return t
        @property
        def op(self):
            return o
        nc.type = type
        nc.op = op
        return nc
    return dec
class Node:
    @property
    def vars(self):
        return self._vars
    def __repr__(self):
        return self.print(self.vars)
    @property
    def show(self):
        return self
    @property
    def int(self):
        return self
    
class NodeArgs(Node):
    @property
    def args(self):
        return self._args
    def __init__(self,args):
        self._args = args
        self._vars = set()
        for x in args:
            self._vars |= x.vars
    def __eq__(self,other):
        return other.op==self.op and self.args==other.args
    
class NodeArgsBinary(NodeArgs):
    @property
    def lhs(self):
        return self._args[0]
    @property
    def rhs(self):
        return self._args[1]
    def print(self,used):
        l = self.args[0].print(used)
        if self.args[0].op not in self.nobrace:
            l = '('+l+')'
        r = self.args[1].print(used)
        if self.args[1].op not in self.nobrace:
            r = '('+r+')'
        return l+self._print_op+r

class Root:
    @property 
    def show(self):
        return self._show
    @property
    def int(self):
        return self._int
    def __eq__(self,other):
        return self.int==other.int
    def __repr__(self):
        return 'Root('+repr(self.show)+')'
    def __init__(self,s,i):
        self._show = s
        self._int = i


In [2]:
#test
x = object()

In [3]:
#test
y=object()

In [4]:
#test
x==y

False

In [5]:
@typeop('rel','or')
class _or(NodeArgsBinary):
    def __init__(self,lhs,rhs):
        assert lhs.type=='rel' and rhs.type=='rel', (lhs,rhs)
        super().__init__([lhs,rhs])
    nobrace = {'ch','not','eq'}
    _print_op = ' or '
def Or(lhs,rhs):
    return Root(_or(lhs.show,rhs.show),
                _or(lhs.int,rhs.int))
    
class _ch(Node):
    @property
    def op(self):
        return 'ch'
    @property
    def type(self):
        return self._type
    @property
    def ch(self):
        return self._ch
    def __init__(self,ch,t):
        self._type = t
        self._ch = ch
        self._vars = {ch}
    def print(self,used):
        return self.ch
    def __eq__(self,och):
        return och.op=='ch' and self.ch==och.ch and self.type==och.type
def RelCh(ch):
    return _ch(ch,'rel')


In [6]:
#test
A = RelCh('A')
A

A

In [7]:
#test
A.show.vars

{'A'}

In [8]:
#test
B = RelCh('B')
Or(A,B)

Root(A or B)

In [9]:
#test
B = RelCh('B')
Or(A,B).int.op

'or'

In [10]:
#test
x = Or(A,B)
x.int.args[0]

A

In [11]:
#test
C = RelCh('C')
Or(A,Or(B,C))

Root(A or (B or C))

In [12]:
#test
A.int==A.int

True

In [13]:
#test
Or(A,B).int==Or(A,B).int

True

In [14]:
#test
Or(A,B)==Or(B,A)

False

In [15]:
@typeop('rel','not')
class _not(NodeArgs):
    @property
    def arg(self):
        return self._args[0]
    nobrace = {'not','ex','an','ch','eq'}
    def __init__(self,arg):
        assert arg.type=='rel'
        super().__init__([arg])
    def print(self,used):
        a = self.args[0].print(used)
        if self.args[0].op not in self.nobrace:
            a = '('+a+')'
        return 'not '+a
def Not(arg):
    return Root(_not(arg.show),
                _not(arg.int))


In [16]:
#test
Not(A)

Root(not A)

In [17]:
#test
Not(Or(A,B))

Root(not (A or B))

In [18]:
#test
Or(Not(A),B)

Root(not A or B)

In [19]:
#test
Not(Not(B))

Root(not not B)

In [20]:
#test
Not(A).int==Not(A).int

True

In [21]:
#test
A.show.vars

{'A'}

In [22]:
@typeop('rel','imp')
class _imp(NodeArgsBinary):
    nobrace = {'ch','or','not','eq'}
    _print_op = ' => '
    def __init__(self,lhs,rhs):
        assert lhs.type=='rel' and rhs.type=='rel'
        super().__init__([lhs,rhs])

def Imp(lhs,rhs):
    return Root(_imp(lhs.show,rhs.show),
                _or(_not(lhs.int),rhs.int))


In [23]:
#test
Imp(A,B).int

not A or B

In [24]:
#test
Imp(A,Or(A,B))

Root(A => A or B)

In [25]:
#test
class foo:
    @staticmethod
    def bar(x):
        f = foo()
        f.__x = x
        return f
    def getx(self):
        return self.__x
a = foo.bar(6)
a.getx()

6

In [26]:
# не изменяет исходных объектов
from copy import copy
def _subst(n,ch,expr):
    assert ch.op=='ch'
    if n.op=='ch':
        if n.ch==ch.ch:
            assert n.type==ch.type
            return expr
        else:
            return n
    elif isinstance(n,bound):
        return n
    else:
        n1 = copy(n)
        n1._args = []
        n1._vars = set()
        for x in n.args:
            y = _subst(x,ch,expr)
            n1._args.append(y)
            n1._vars|=y.vars
        return n1
def subst(a,ch,expr):
    return Root(_subst(a.show,ch.int,expr.show),
                _subst(a.int ,ch.int,expr.int))

In [27]:
#test
A.show.vars

{'A'}

In [28]:
class NodeArgsBound(NodeArgs):
    @property
    def bound(self):
        return self._bound
    @property
    def rel(self):
        return self._args[0]
    def __init__(self,ch,R,rec = []):
        assert R.type=='rel'
        assert isinstance(ch,_ch),type(ch)
        assert ch.type=='term',ch
        self._bound = bound([ch.ch]+rec)
        super().__init__([_subst(R,ch,self.bound)])
    def __eq__(self,other):
        if other.op!=self.op: return False
        self.bound._cureq = other.bound
        other.bound._cureq = self
        try:
            rez = self.args==other.args
        finally:
            self.bound._cureq = None
            other.bound._cureq = None
        return rez


In [29]:
@typeop('term','bound')
class bound(Node):
    _default = 'x,y,z,a,b,c,d,e,f,g,h,i,j,k,l,m,n'.split(',')
    def __init__(self,rec):
        self._recommended = rec
        self._vars = set()
    def choose(self,used):
        for ch in self._recommended:
            if ch not in used:
                self._choosen = ch
                return
        for ch in self._default:
            if ch not in used:
                self._choosen = ch
                return
        raise Error("bound can't choose")
    def print(self,used):
        return self._choosen
    def __eq__(self,other):
        return self is other or other is self._cureq
    
def TermCh(ch):
    return _ch(ch,'term')
        
@typeop('term','tau')
class _tau(NodeArgsBound):
    def print(self,used):
        self.bound.choose(used|self.vars)
        #print(self.bound.choosen,type(self.bound.choosen))
        u = used|{self.bound._choosen}
        return 'tau('+self.bound.print(u)+','+\
                self.args[0].print(u)+')'
    
def Tau(ch,R,rec = []):
    return Root(_tau(ch.int,R.show,rec),
                _tau(ch.int,R.int,rec))


In [30]:
#test
x = TermCh('x')
y = TermCh('y')
x

x

In [31]:
#test
t = Tau(x,A)

In [32]:
#test
t = Tau(y,A)

In [33]:
#test
t

Root(tau(y,A))

In [34]:
#test
Tau(x,A).int==Tau(y,A).int

True

In [35]:
#test
Tau(x,A).int==Tau(x,B).int

False

In [36]:
@typeop('rel','eq')
class _eq(NodeArgsBinary):
    nobrace = {'ch','tau','bound'}
    _print_op = '=='
    def __init__(self,lhs,rhs):
        assert lhs.type=='term' and rhs.type=='term'
        super().__init__([lhs,rhs])
def Eq(lhs,rhs):
    return Root(_eq(lhs.show,rhs.show),
                _eq(lhs.int,rhs.int))


In [37]:
#test
Eq(x,y)

Root(x==y)

In [38]:
#test
Tau(x,Eq(x,y))

Root(tau(x,x==y))

In [39]:
#test
Eq(x,Tau(x,Eq(x,y)))

Root(x==tau(z,z==y))

In [40]:
@typeop('rel','ex')
class _ex(NodeArgsBound):
    def print(self,used):
        self.bound.choose(used|self.vars)
        #print(self.bound.choosen,type(self.bound.choosen))
        u = used|{self.bound._choosen}
        return 'Ex '+self.bound.print(u)+':'+\
                self.args[0].print(u)

def Ex(ch,R,rec = []):
    return Root(_ex(ch.int,R.show,rec),
                _subst(R.int,ch.int,_tau(ch.int,R.int,rec)))


In [41]:
#test
Or(Ex(x,Eq(x,y)),Eq(x,x))

Root((Ex z:z==y) or x==x)

In [42]:
#test
Ex(x,Eq(x,y)).int

tau(x,x==y)==y

In [43]:
@typeop('rel','an')
class _an(NodeArgsBound):
    def print(self,used):
        self.bound.choose(used|self.vars)
        #print(self.bound.choosen,type(self.bound.choosen))
        u = used|{self.bound._choosen}
        return 'An '+self.bound.print(u)+':'+\
                self.args[0].print(u)
def An(ch,R,rec = []):
    return Root(_an(ch.int,R.show,rec),
                _not(Ex(ch,Not(R),rec).int))


In [44]:
#test
An(x,Eq(x,y))

Root(An x:x==y)

In [45]:
#test
An(x,Eq(x,y)).int==An(x,Eq(x,y)).int

True

In [46]:
#test
Eq(x,y).int==Eq(x,y).int

True

In [47]:
#test
An(x,Eq(x,y)).int

not not tau(x,not x==y)==y

In [48]:
#test
An(x,A).int

not not A

In [53]:
# класс доказанного утверждепия
#   утверждение//rel
#   получение из доказанного
#     подстановкой
#     C1
#     совпадает внут.структ.
#     S5
#     S6
#     S7
#     S8
#     создание аксиомы
#   ... ссылки на предков и потомков
class Prove:
    @property
    def rel(self):
        return self.__rel
    @property
    def show(self):
        return self.rel.show
    @property
    def int(self):
        return self.rel.int
    def __repr__(self):
        return repr(self.rel)
    def __eq__(other):
        return self.rel==other.rel
    @staticmethod
    def __make(R):
        ps = Prove()
        ps.__rel = R
        return ps
        
    # методы доказателства
    @staticmethod
    def same(_from,_to):
        assert isinstance(_from,Prove)
        assert _from.__rel.int==_to.int
        return Prove.__make(_to)
    @staticmethod
    def subst(p,ch,expr):
        assert isinstance(p,Prove)
        return Prove.__make(subst(copy(p.__rel),ch,expr))
    @staticmethod
    def C1(_from,_because,_to = None):
        assert isinstance(_from,Prove) and isinstance(_because,Prove)
        _from = _from.__rel.int
        _because = _because.__rel.int
        assert _because.op=='or' and _because.lhs.op=='not'
        assert _because.lhs.arg==_from
        if _to is None:
            _to = _because.rhs
        else:
            assert _because.rhs==_to.int
        return Prove.__make(_to)
    @staticmethod
    def S1(A=None):
        if A is None:
            A = RelCh('A')
        return Prove.__make(Imp(Or(A,A),A))
    @staticmethod
    def S2(A=None,B=None):
        if B is None:
            assert A is None
            A = RelCh('A')
            B = RelCh('B')
        return Prove.__make(Imp(A,Or(A,B)))
    @staticmethod
    def S3(A=None,B=None):
        if B is None:
            assert A is None
            A = RelCh('A')
            B = RelCh('B')
        return Prove.__make(Imp(Or(A,B),Or(B,A)))
    @staticmethod
    def S4(A=None,B=None,C=None):
        if C is None:
            assert A is None
            A = RelCh('A')
            B = RelCh('B')
            C = RelCh('C')
        return Prove.__make(Imp(Imp(A,B),Imp(Or(C,A),Or(C,B))))
    @staticmethod
    def S5(R,x,T):
        return Prove.__make(Imp(subst(R,x,T),Ex(x,R)))
    @staticmethod
    def S6(R,x,T,U):
        return Prove.__make(
            Imp(Eq(T,U),Equiv(subst(R,x,T),subst(R,x,U))))
    @staticmethod
    def S7(R,S,x):
        return Prove.__make(
            Imp(An(x,Equiv(R,S)),Eq(Tau(x,R),Tau(x,S))))
        

In [54]:
#test
class C1:
    pass
class C2(C1):
    pass
x = C2()
isinstance(x,C1)

True

In [56]:
#test
!jupyter nbconvert Nodes.ipynb --to python
inf = open('Nodes.py')
outf = open('Nodes1.py','w')
test = False
for line in inf:
    if test:
        outf.write('#test  '+line)
        if line=='\n':
            #print(repr(line))
            test = False
    else:
        if line.startswith('#test'):
            test = True
        outf.write(line)
outf.close()
inf.close()
!move Nodes1.py Nodes.py

[NbConvertApp] Converting notebook Nodes.ipynb to python
[NbConvertApp] Writing 12248 bytes to Nodes.py


ЏҐаҐ¬ҐйҐ­® д ©«®ў:         1.


In [52]:
#test
def dec(C):
    C.x=5
    return C
@dec
class cl:
    pass
cl.x

5